In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [6]:
strade_principali = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/strade_principali_piemonte/strade_principali_piemonte.shp').to_crs(epsg=3857)
rete_stradale = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/strade.geojson').to_crs(epsg=3857)

In [17]:
strade_principali

,GP_RTP,GP_RCY,Shape_Leng,geometry
0,4,380,0.016881,"LINESTRING (901127.078 5593675.513, 901150.619..."
1,4,380,0.054220,"LINESTRING (903436.099 5596337.345, 903453.34 ..."
2,3,380,0.027438,"LINESTRING (900836.851 5590749.088, 900740.071..."
3,3,380,0.020449,"LINESTRING (900973.823 5590811.431, 900999.534..."
4,3,380,0.036795,"LINESTRING (901192.359 5593502.832, 901159.09 ..."
...,...,...,...,...
34041,3,380,0.000068,"LINESTRING (911489.897 5725219.027, 911490.821..."
34042,3,380,0.000240,"LINESTRING (911517.422 5725226.846, 911490.821..."
34043,3,380,0.000622,"LINESTRING (911490.821 5725229.729, 911486.391..."
34044,1,380,0.007936,"LINESTRING (949038.507 5735534.089, 949207.126..."


In [13]:
strade_principali.drop(columns=['GP_REX', 'GP_RAV', 'GP_RRG', 'GP_RSE', 'GP_RSI', 'GP_RSY', 'gp_gripreg'], inplace=True)

GP_RTP

0.	Unspecified
1.	Highways
2.	Primary roads
3.	Secondary roads
4.	Tertiary roads
5.	Local roads

In [19]:
from shapely.geometry import MultiPolygon

road_width = {1: 14,
              2: 7,
              3: 5,
              4: 3,
              5: 2.5,
              0: 2.5}

def buffer_linestring(x, width):
    if x.geometry.geom_type == 'LineString':
        return x.geometry.buffer(width)
    elif x.geometry.geom_type == 'MultiLineString':
        buffers = [line.buffer(width) for line in x.geometry.geoms]
        return MultiPolygon(buffers)

strade_principali['geometry'] = strade_principali.apply(lambda x: buffer_linestring(x, road_width[x.loc['GP_RTP']]), axis=1)

strade_principali.geom_type.unique()

array(['Polygon', 'MultiPolygon'], dtype=object)

In [28]:
strade_principali.drop(columns=['GP_RCY', 'Shape_Leng'], inplace=True)

In [30]:
strade_principali.to_file('/nfs/home/genovese/thesis-wildfire-genovese/database/strade.geojson', index=False, driver='GeoJSON')

---